In [ ]:
import os
import requests
import json
import time
import pandas as pd
from copy import deepcopy
from itertools import combinations

### Verify connection to the API

In [ ]:
api_url = 'https://windfarmer.dnv.com/api/v2/'
auth_token = os.environ['WINDFARMER_ACCESS_KEY']
headers = {
    'Authorization': f'Bearer {auth_token}',
    'Content-Type': 'application/json'
}
response = requests.get(api_url + 'Status', headers = headers)
print(f'Response from Status: {response.status_code}')
print(response.text)

#  **Part 1** - generating a demo batch of API inputs [OPTIONAL, skip if you have generated the batch otherwise] 

Let's imagine we are asked to remove three turbines from a layout in the most energy efficient way. We'll do that by identifying the worst 10 turbines in the layout and later evaulating the yield of a batch of alternative layouts, each created by removing 3 different turbines from the set of 10 worst producing locations.  

Initial calculation to get a feel where the most heavily waked zones are 

In [ ]:
# load the input json for The Bowl
with open('../../../DemoData/TheBowl/TheBowl.json', 'r') as f:
    json_string = f.read()
    input_data = json.loads(json_string)
    
start = time.time()
response = requests.post(
    api_url + 'AnnualEnergyProduction', 
    headers=headers,
    json = input_data)
    
print(f'Response {response.status_code} - {response.reason} in {time.time() - start:.2f}s')
# Print the error detail if we haven't receieved a 200 OK response
if response.status_code != 200:
    print(json.loads(response.content)['detail'])

identifting the worst 10 turbines

In [ ]:
result = json.loads(response.content)
yields = [x['fullAnnualYield_MWh_per_year'] for x in result['windFarmAepOutputs'][0]['turbineResults']]
labels = [x['turbineName'] for x in result['windFarmAepOutputs'][0]['turbineResults']]
yields_df = pd.DataFrame.from_dict({k:v for k,v in zip(labels, yields)}, orient='index', columns=['Yield (MWh/year)'])
yields_df.sort_values(by='Yield (MWh/year)', ascending=True, inplace=True)
worst_10_turbines = list(yields_df.iloc[:10].index)
print(worst_10_turbines)

generating all the possible combinations of removing two out of ten candidate turbines

In [ ]:
turbine_removal_alternatives = list(combinations(worst_10_turbines, 3))
print(turbine_removal_alternatives)

generate API inputs for each alternative

In [ ]:
for alternative in turbine_removal_alternatives:
    temp_json = deepcopy(input_data)
    for turbine in input_data["windFarms"][0]["turbines"]:
        if turbine["name"] in alternative:
            temp_json["windFarms"][0]["turbines"].remove(turbine)
    if not os.path.exists('./InputBatch'):
        os.makedirs('./InputBatch')
    with open('./InputBatch/{0}.json'.format('_'.join(alternative)), 'w') as f:
        f.write(json.dumps(temp_json, indent=4))

#  **Part 2** - batch processing [if you're using this for API inputs generated outside of the script, put them in a folder `InputBatch`, next to the script]

### **Define user inputs**

In [ ]:
# specify the desired model combination
WAKE_MODEL_CHOICE = "EddyViscosity" # EddyViscosity/ModifiedPark/TurbOPark/CFDML
BLOCKAGE_MODEL_CHOICE = "BEET" # BEET/CFDML
CALCULATE_EFFICIENCIES = True #True/False 
  
# specify the path to the input json files
PATH_TO_INPUTS = './InputBatch'

**Note**: the part responsible for model settings in the input jsons contained in the `./InputBatch` directory will get overwritten by the settings specified in the code cell above  

In [ ]:
# the below dictionaries contain various settings, of which we make a selection for the particular run  (inside set_model_settings)
lwf_paramters = { # LWF's default offshore settings
                    "baseRoughnessZ01": 0.0002,
                    "increasedRoughnessZ02": 0.0192,
                    "geometricWidthDiameters": 1.0,
                    "recoveryStartDiameters": 120.0,
                    "fiftyPercentRecoveryDiameters": 40.0
                }
wake_models = {"EddyViscosity": {"model_key":"eddyViscosity", "model_settings":{"useLargeWindFarmModel": True, "largeWindFarmCorrectionParameters": lwf_paramters}},
               "ModifiedPark":  {"model_key":"modifiedPark", "model_settings": {"useLargeWindFarmModel": True, "largeWindFarmCorrectionParameters": lwf_paramters}},
               "TurbOPark": {"model_key":"turbOPark", "model_settings": {"wakeExpansion": 0.04}},
               "CFDML": {"model_key":"cfdml", "model_settings":{"gnnType": "Offshore", "gnnStabilityClass": "UnstableNeutral"}}}
blockage_models = {"BEET": {"model_key":"beet", "model_settings":{"significantAtmosphericStability": False,
                                                                  "inclusionOfNeighborsBufferZoneInMeters": 1000.0,
                                                                  "blockageCorrectionApplicationMethod": "OnWindSpeed"}},
                   "CFDML": {"model_key":"cfdml", "model_settings":{"cfdmlSettings": {"gnnType": "Offshore", "gnnStabilityClass": "UnstableNeutral"}, 
                                                                    "blockageCorrectionApplicationMethod": "OnWindSpeed",
                                                                    "cfdmlBlockageWindSpeedDependency": "FromBlockageExtrapolationCurve"}}}
# the below functions wrap steps used more than once in later code-cells
def switch_off_fpm_export(input_json):
    # this function makes sure fpm export is switched off - to speed up API response time
    for fpm in input_json["energyEfficienciesSettings"]["turbineFlowAndPerformanceMatrixOutputSettings"].keys():
        if fpm != "localTurbineWindSpeedsOutputSettings":
            input_json["energyEfficienciesSettings"]["turbineFlowAndPerformanceMatrixOutputSettings"][fpm] = False
        else:
            input_json["energyEfficienciesSettings"]["turbineFlowAndPerformanceMatrixOutputSettings"][fpm] = None

def set_model_settings(input_json):
    # let's set the modeling options contained in the json
    input_json["energyEfficienciesSettings"]["calculateEfficiencies"] = CALCULATE_EFFICIENCIES
    # pull the relevant default settings from the dicts predefined above
    input_json["energyEfficienciesSettings"]["wakeModel"]["wakeModelType"] = WAKE_MODEL_CHOICE
    input_json["energyEfficienciesSettings"]["wakeModel"][wake_models[WAKE_MODEL_CHOICE]["model_key"]] = wake_models[WAKE_MODEL_CHOICE]["model_settings"]
    input_json["energyEfficienciesSettings"]["blockageModel"]["blockageModelType"] = BLOCKAGE_MODEL_CHOICE
    input_json["energyEfficienciesSettings"]["blockageModel"][blockage_models[BLOCKAGE_MODEL_CHOICE]["model_key"]] = blockage_models[BLOCKAGE_MODEL_CHOICE]["model_settings"]
    switch_off_fpm_export(input_json)

def submit_the_job(input_json_path, job_id_dict):
    # load the json inputs 
    with open(input_json_path) as f:
        json_string = f.read()
        input_json = json.loads(json_string)
    # override the modeling settings pre-exisitng in the json with the ones specified above
    set_model_settings(input_json)
    # save the updated json
    with open(input_json_path, 'w') as f:
        f.write(json.dumps(input_json, indent=4))
    start = time.time()
    response = requests.post(
        api_url + 'AnnualEnergyProductionAsync', 
        headers=headers,
        json = input_json
    )
    print(f'Response {response.status_code} - {response.reason} in {time.time() - start:.2f}s')
    response_json = json.loads(response.content)
    if response.status_code == 202:
        job_id_dict[os.path.basename(input_json_path)] = response_json['jobId']
        print("job ID: " + str(response_json['jobId']))
    return job_id_dict

def poll_results(job_id_dict):
    if not os.path.exists(f'./Results/'):
        os.mkdir(f'./Results/')
    jobs_completed_dict = {k:False for k in job_id_dict.keys()}
    jobs_failed_list = []
    while not all(jobs_completed_dict.values()):
        for job_filename, job_id in job_id_dict.items():
            if not jobs_completed_dict[job_filename]:
                response = requests.get(api_url + 'AnnualEnergyProductionAsync', headers = headers, params={'jobId':job_id})
                response_json = json.loads(response.content)
                if response_json['status'] == 'SUCCESS':
                    jobs_completed_dict[job_filename] = True
                    print(f'Job {job_filename} is completed')
                    with open(f'./Results/{job_filename}', 'w') as f:
                        f.write(json.dumps(response_json, indent=4))
                elif response_json['status'] == 'FAILED':
                    print(f'Job {job_id} has FAILED.')
                    jobs_failed_list.append(job_filename)
                    jobs_completed_dict[job_filename] = True
                else:
                    print(f'Job {job_filename} is {response_json["status"]}, JobID = {job_id}')
        print("No. of completed jobs: " + str(sum(jobs_completed_dict.values())) + " out of " + str(len(jobs_completed_dict)))
        print("Among the completed jobs {} failed.".format(len(jobs_failed_list)))
        time.sleep(5)
    return jobs_completed_dict, jobs_failed_list

submit the batch of all simulation cases to the async endpoint of the API

In [ ]:
# submit the jobs
job_id_dict = {}
for input_file in os.listdir(PATH_TO_INPUTS):
    job_id_dict = submit_the_job(os.path.join(PATH_TO_INPUTS,input_file),job_id_dict)

# persist the job_ids to disc
with open('job_id_dict.json', 'w') as f:
    f.write(json.dumps(job_id_dict, indent=4))

## At this point you can go get a coffee or lunch, or even switch the PC off. 

WindFarmerServices API will compute and store your results for 7 days. The subsequent cells read the job id's from file so execution of the notebook may be resumed at any point in time.

Poll the API to see if all results are available...

In [ ]:
# load the job ids submitted
with open('job_id_dict.json', 'r') as f:
    job_id_dict = json.load(f)
# poll the API for results, until all jobs are completed
jobs_completed_dict, jobs_failed_list = poll_results(job_id_dict)

In case some jobs failed (this may happen and not necessarily be linked with your inputs), let's re-submit jobs that did fail...

In [ ]:
if len(jobs_failed_list) == 0:
    print("All jobs completed successfully, nothing to re-submit.")
# re-submit failed jobs once
resubmitted_job_id_dict = {}
for job_file in jobs_failed_list:
    print(f'Resubmitting job {job_file}')
    resubmitted_job_id_dict = submit_the_job(os.path.join(PATH_TO_INPUTS,job_file), resubmitted_job_id_dict)
# persist the job_ids to disc
with open('resubmitted_job_id_dict.json', 'w') as f:
    f.write(json.dumps(resubmitted_job_id_dict, indent=4))
# and you can go get coffee again if needed...   

... and poll for the re-submitted results

In [ ]:
if len(jobs_failed_list) == 0:
    print("No jobs re-submitted, so nothing to poll for.")
# load the job ids submitted
with open('resubmitted_job_id_dict.json', 'r') as f:
    resubmitted_job_id_dict = json.load(f)
# poll the API for results, until all jobs are completed
jobs_completed_dict, jobs_failed_list = poll_results(resubmitted_job_id_dict)

# **Part 3** Process the resulting jsons and extract the relevant numbers into a `DataFrame`

In [ ]:
case_summary=[]
for results_file in os.listdir('./Results'):
    with open('./Results/'+results_file) as f:
        json_string = f.read()
        results_json = json.loads(json_string)
    subject_farm_results_dict = results_json["results"]["windFarmAepOutputs"][0]
    if CALCULATE_EFFICIENCIES:
        case_summary.append({
            "Case name": results_file,
            "Gross Yield [MWh/Annum]": subject_farm_results_dict["grossAnnualEnergyYield_MWh_per_year"],
            "Blockage efficiency [%]": subject_farm_results_dict["blockageOnAnnualEnergyYield_MWh_per_year"] / subject_farm_results_dict["grossAnnualEnergyYield_MWh_per_year"],
            "Internal wake efficiency [%]": subject_farm_results_dict["internalWakesOnAnnualEnergyYield_MWh_per_year"] / subject_farm_results_dict["blockageOnAnnualEnergyYield_MWh_per_year"],
            "Hysteresis efficiency [%]": subject_farm_results_dict["hysteresisAdjustmentOnAnnualEnergyYield_MWh_per_year"] / subject_farm_results_dict["internalWakesOnAnnualEnergyYield_MWh_per_year"],
            "Internal LWF efficiency [%]": subject_farm_results_dict["largeWindFarmCorrectionOnAnnualEnergyYield_MWh_per_year"] / subject_farm_results_dict["hysteresisAdjustmentOnAnnualEnergyYield_MWh_per_year"],
            "External wake efficiency [%]": subject_farm_results_dict["neighborsWakesOnAnnualEnergyYield_MWh_per_year"] / subject_farm_results_dict["largeWindFarmCorrectionOnAnnualEnergyYield_MWh_per_year"],
            "Full Yield [MWh/Annum]": subject_farm_results_dict["fullAnnualEnergyYield_MWh_per_year"],
        })
    else:
        case_summary.append({
            "Case name": results_file,
            "Gross Yield [MWh/Annum]": subject_farm_results_dict["grossAnnualEnergyYield_MWh_per_year"],
            "Full Yield [MWh/Annum]": subject_farm_results_dict["fullAnnualEnergyYield_MWh_per_year"],
        })
case_summary_df = pd.DataFrame.from_dict(case_summary)
case_summary_df.sort_values(by='Full Yield [MWh/Annum]', ascending=True, inplace=True)
case_summary_df['Yield uplift relative to worst choice [%]'] = round(case_summary_df['Full Yield [MWh/Annum]'] / case_summary_df['Full Yield [MWh/Annum]'].min() * 100 - 100,3)
case_summary_df

In [ ]:
print("best choice -> remove turbines: " + case_summary_df.iloc[-1]['Case name'].replace('.json','').replace('_',','))

How to read further values from the results can be determined from the API documentation, or look at the JSON files in your results folder.

https://myworkspace.dnv.com/download/public/renewables/windfarmer/manuals/latest/WebAPI/ReleaseNotes/releaseNotes.html#openapi-specifications 